In [1]:
import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px


pd.set_option('display.max_columns',None)
import psycopg2


#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Algorithms to use
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

import warnings
warnings.filterwarnings("ignore")

#importing PCA and TSNE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [17]:
# Specify the path to your text file
file_path = 'C:/Users/GenepoweRx_Madhu/Downloads/krishna_covered_pos/isglGvYQxrwjOrFL.txt'

# Read the text file into a DataFrame
data = pd.read_csv(file_path, delimiter='\t')
data

,#Uploaded_variation,Location,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,SYMBOL_SOURCE,HGNC_ID,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,REFSEQ_MATCH,REFSEQ_OFFSET,GIVEN_REF,USED_REF,BAM_EDIT,SIFT,PolyPhen,AF,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS
0,rs12248560,10:94761900-94761900,A,upstream_gene_variant,MODIFIER,CYP2C19,1557,Transcript,NM_000769.4,protein_coding,-,-,-,-,-,-,-,-,-,"rs12248560,CR067132",781,1,-,EntrezGene,-,ENST00000371321.9,-,-,-,-,-,C,C,-,-,-,-,-,-,"1,1","24944790,30593137,17048007,22879966,22992668,2...",-,-,-,-,-
1,rs12248560,10:94761900-94761900,T,upstream_gene_variant,MODIFIER,CYP2C19,1557,Transcript,NM_000769.4,protein_coding,-,-,-,-,-,-,-,-,-,"rs12248560,CR067132",781,1,-,EntrezGene,-,ENST00000371321.9,-,-,-,-,-,C,C,-,-,-,0.1532,"drug_response,benign",-,"1,1","24944790,30593137,17048007,22879966,22992668,2...",-,-,-,-,-
2,rs28399504,10:94762706-94762706,G,start_lost,HIGH,CYP2C19,1557,Transcript,NM_000769.4,protein_coding,1/9,-,-,-,26,1,1,M/V,Atg/Gtg,"rs28399504,CM983291",-,1,-,EntrezGene,-,ENST00000371321.9,-,-,-,-,-,A,A,-,deleterious_low_confidence(0.01),probably_damaging(0.998),0.0008,"drug_response,likely_benign",-,"1,1","24944790,22992668,29193749,26857559,18547414,2...",-,-,-,-,-
3,rs28399504,10:94762706-94762706,T,start_lost,HIGH,CYP2C19,1557,Transcript,NM_000769.4,protein_coding,1/9,-,-,-,26,1,1,M/L,Atg/Ttg,"rs28399504,CM983291",-,1,-,EntrezGene,-,ENST00000371321.9,-,-,-,-,-,A,A,-,deleterious_low_confidence(0),probably_damaging(0.997),-,-,-,"1,1","24944790,22992668,29193749,26857559,18547414,2...",-,-,-,-,-
4,rs55752064,10:94762755-94762755,A,missense_variant,MODERATE,CYP2C19,1557,Transcript,NM_000769.4,protein_coding,1/9,-,-,-,75,50,17,L/H,cTt/cAt,rs55752064,-,1,-,EntrezGene,-,ENST00000371321.9,-,-,-,-,-,T,T,-,deleterious_low_confidence(0),probably_damaging(0.98),-,-,-,1,"21358751,22027650,34621706",-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,rs2740574,7:99784473-99784473,G,upstream_gene_variant,MODIFIER,CYP3A4,1576,Transcript,NM_001202855.3,protein_coding,-,-,-,-,-,-,-,-,-,"rs2740574,CR984104",289,-1,-,EntrezGene,-,-,-,-,-,-,-,C,C,-,-,-,-,-,-,"1,1",-,-,-,-,-,-
649,rs2740574,7:99784473-99784473,T,upstream_gene_variant,MODIFIER,CYP3A4,1576,Transcript,NM_001202855.3,protein_coding,-,-,-,-,-,-,-,-,-,"rs2740574,CR984104",289,-1,-,EntrezGene,-,-,-,-,-,-,-,C,C,-,-,-,0.7692,benign,-,"1,1",-,-,-,-,-,-
650,rs2740574,7:99784473-99784473,A,upstream_gene_variant,MODIFIER,CYP3A4,1576,Transcript,NM_017460.6,protein_coding,-,-,-,-,-,-,-,-,-,"rs2740574,CR984104",289,-1,-,EntrezGene,-,ENST00000651514.1,-,-,-,-,-,C,C,-,-,-,-,-,-,"1,1",-,-,-,-,-,-
651,rs2740574,7:99784473-99784473,G,upstream_gene_variant,MODIFIER,CYP3A4,1576,Transcript,NM_017460.6,protein_coding,-,-,-,-,-,-,-,-,-,"rs2740574,CR984104",289,-1,-,EntrezGene,-,ENST00000651514.1,-,-,-,-,-,C,C,-,-,-,-,-,-,"1,1",-,-,-,-,-,-


In [33]:
# Specify the path to your text file
file_path = 'C:/Users/GenepoweRx_Madhu/Downloads/krishna_covered_pos/isglGvYQxrwjOrFL.txt'

# Read the text file into a DataFrame
df = pd.read_csv(file_path, delimiter='\t')
df = df[['#Uploaded_variation', 'Location', 'USED_REF', 'Allele', 'SYMBOL']]
df

,#Uploaded_variation,Location,USED_REF,Allele,SYMBOL
0,rs12248560,10:94761900-94761900,C,A,CYP2C19
1,rs12248560,10:94761900-94761900,C,T,CYP2C19
2,rs28399504,10:94762706-94762706,A,G,CYP2C19
3,rs28399504,10:94762706-94762706,A,T,CYP2C19
4,rs55752064,10:94762755-94762755,T,A,CYP2C19
...,...,...,...,...,...
648,rs2740574,7:99784473-99784473,C,G,CYP3A4
649,rs2740574,7:99784473-99784473,C,T,CYP3A4
650,rs2740574,7:99784473-99784473,C,A,CYP3A4
651,rs2740574,7:99784473-99784473,C,G,CYP3A4


In [34]:
df['CHROM'] = 'chr' + df['Location'].str.split(':').str[0]
df['Start'] = df['Location'].str.split(':').str[1].str.split('-').str[0].astype(int)
df['End'] = df['Location'].str.split(':').str[1].str.split('-').str[1].astype(int)
df.rename(columns={'#Uploaded_variation': 'rsid', 'USED_REF':'REF', 'Allele':'ALT', 'SYMBOL':'Gene'},inplace=True)
df = df[['rsid', 'Gene', 'CHROM', 'Start', 'End', 'REF', 'ALT']]
df

,rsid,Gene,CHROM,Start,End,REF,ALT
0,rs12248560,CYP2C19,chr10,94761900,94761900,C,A
1,rs12248560,CYP2C19,chr10,94761900,94761900,C,T
2,rs28399504,CYP2C19,chr10,94762706,94762706,A,G
3,rs28399504,CYP2C19,chr10,94762706,94762706,A,T
4,rs55752064,CYP2C19,chr10,94762755,94762755,T,A
...,...,...,...,...,...,...,...
648,rs2740574,CYP3A4,chr7,99784473,99784473,C,G
649,rs2740574,CYP3A4,chr7,99784473,99784473,C,T
650,rs2740574,CYP3A4,chr7,99784473,99784473,C,A
651,rs2740574,CYP3A4,chr7,99784473,99784473,C,G


In [40]:
grouped = df.groupby(['rsid', 'CHROM', 'Start', 'End', 'REF'])['ALT'].agg(ALT=lambda x: ','.join(set(x))).reset_index()
grouped

,rsid,CHROM,Start,End,REF,ALT
0,rs1024323,chr4,3004316,3004316,C,"G,T,A"
1,rs10264272,chr7,99665212,99665212,-,T
2,rs10264272,chr7,99665212,99665212,C,T
3,rs1057910,chr10,94981296,94981296,A,"G,C"
4,rs1058164,chr22,42129130,42129130,C,G
...,...,...,...,...,...,...
126,rs7900194,chr10,94942309,94942309,G,"C,A,T"
127,rs79292917,chr22,42127852,42127852,C,T
128,rs8192709,chr19,40991369,40991369,C,T
129,rs9332131,chr10,94949282,94949283,A,-


In [41]:
grouped.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/krishna_covered_pos/Grouped_pos.xlsx', index=False)

In [42]:
df = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/krishna_covered_pos/Grouped_pos.xlsx')
df

,rsid,CHROM,Start,End,REF,ALT
0,rs1024323,chr4,3004316,3004316,C,"G,T,A"
1,rs10264272,chr7,99665212,99665212,C,T
2,rs1057910,chr10,94981296,94981296,A,"G,C"
3,rs1058164,chr22,42129130,42129130,C,G
4,rs1058930,chr10,95058362,95058362,G,"C,A"
...,...,...,...,...,...,...
123,rs7900194,chr10,94942309,94942309,G,"C,A,T"
124,rs79292917,chr22,42127852,42127852,C,T
125,rs8192709,chr19,40991369,40991369,C,T
126,rs9332131,chr10,94949282,94949283,A,-


In [43]:
df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/krishna_covered_pos/Haplo_data.xlsx')
df_1

,Haplotype,Gene,rsid
0,UGT1A3*6,UGT1A3,rs45449995
1,UGT1A1*1,UGT1A1,NaN
2,UGT1A1*28,UGT1A1,rs3064744
3,SLCO1B1*37,SLCO1B1,rs2306283
4,SLCO1B1*5,SLCO1B1,rs4149056
...,...,...,...
134,TPMT*3A,TPMT,rs1142345
135,TPMT*3C,TPMT,rs1142345
136,TPMT*3A,TPMT,rs1800460
137,TPMT*3A,TPMT,rs1142345


In [44]:
merged_haplo = pd.merge(df_1, df, on = 'rsid', how = 'left', sort = False)
merged_haplo

,Haplotype,Gene,rsid,CHROM,Start,End,REF,ALT
0,UGT1A3*6,UGT1A3,rs45449995,chr2,233729934.0,233729934.0,A,G
1,UGT1A1*1,UGT1A1,NaN,NaN,NaN,NaN,NaN,NaN
2,UGT1A1*28,UGT1A1,rs3064744,chr2,233760234.0,233760238.0,ATATATATATATATA,"ATATATATATATA,ATATATATATATATATA,ATATATATATA,AT..."
3,SLCO1B1*37,SLCO1B1,rs2306283,chr12,21176804.0,21176804.0,A,"G,C,T"
4,SLCO1B1*5,SLCO1B1,rs4149056,chr12,21178615.0,21178615.0,T,C
...,...,...,...,...,...,...,...,...
134,TPMT*3A,TPMT,rs1142345,chr6,18130687.0,18130687.0,T,"G,C"
135,TPMT*3C,TPMT,rs1142345,chr6,18130687.0,18130687.0,T,"G,C"
136,TPMT*3A,TPMT,rs1800460,chr6,18138997.0,18138997.0,C,T
137,TPMT*3A,TPMT,rs1142345,chr6,18130687.0,18130687.0,T,"G,C"


In [45]:
merged_haplo.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/krishna_covered_pos/Haplo_pos_mapped.xlsx', index = False)